In [97]:
using FunctionalCollections
using Iterators
using Pipe
using Compat

macro printval(ee)
    ee_expr = @sprintf "%s" string(ee)
    esc(:(println($ee_expr," = ", $ee)))
end

macro pz(ee)
    ee_expr = @sprintf "%s" string(ee)
    esc(:(println($ee_expr,"\t\t",typeof($ee), "\t", size($ee))))
end

push!(LOAD_PATH, ".")
push!(LOAD_PATH, "../util/")

6-element Array{ByteString,1}:
 "/home/ubuntu/build/julia-master/usr/local/share/julia/site/v0.5"
 "/home/ubuntu/build/julia-master/usr/share/julia/site/v0.5"      
 "."                                                              
 "../util/"                                                       
 "."                                                              
 "../util/"                                                       

In [2]:
addprocs(11)

11-element Array{Int64,1}:
  2
  3
  4
  5
  6
  7
  8
  9
 10
 11
 12

In [3]:
@everywhere using Distances

In [ ]:

@everywhere data = open("doctors_letters.jsz","r") do fh
    deserialize(fh)    
end
test_set = data["test_set"];
data

In [4]:
@everywhere data = open("atis_data2.jsz","r") do fh
    deserialize(fh)    
end
test_set = data["test_set"];
data

Dict{ASCIIString,Any} with 7 entries:
  "zeroed_words"  => ASCIIString[]
  "length_prob"   => [0.00273293,0.0141963,0.0332591,0.054354,0.0725396,0.08505…
  "LL"            => 300x1126 Array{Float64,2}:…
  "indexed_words" => AbstractString["the","of","to","and","in","a","for","that"…
  "test_set"      => [SubString{ASCIIString}["what","is","the","cheapest","way"…
  "trigrams"      => Any[("**START1**","**START2**","what"),("**START2**","what…
  "word_indexes"  => Dict{AbstractString,Int64}("carried"=>452,"diabetic"=>1069…

In [6]:
@everywhere data = open("brown_data.jsz","r") do fh
    deserialize(fh)    
end
test_set = data["test_set"];
data

Dict{ASCIIString,Any} with 7 entries:
  "zeroed_words"  => ASCIIString[]
  "length_prob"   => [0.00631022,0.013634,0.0204009,0.0262119,0.0309737,0.03471…
  "LL"            => 300x40482 Array{Float64,2}:…
  "indexed_words" => AbstractString["the",",",".","of","to","and","in","a","for…
  "test_set"      => [ASCIIString["``","only","a","relative","handful","of","su…
  "trigrams"      => Any[("**START1**","**START2**","edison"),("**START2**","ed…
  "word_indexes"  => Dict{AbstractString,Int64}("baleful"=>34074,"confined"=>77…

In [9]:
@everywhere function lookup_sowe(data, sent::AbstractString)
    lookup_sowe(data, sent |> split)
end

@everywhere function lookup_sowe{S<:AbstractString}(data, sent::Vector{S})
    sum([data["LL"][:,data["word_indexes"][word]] for word in sent]) 
end

@everywhere function lookup_words(data, path)
    [data["indexed_words"][ii] for ii in path]
end

In [10]:
@everywhere const ϵ = 10.0^-6

In [85]:
@everywhere @inline function get_end(LL, ws::Vector{Int64})
    @inbounds sofar = length(ws)>0 ? sum([LL[:,ii] for ii in ws]) : zeros(LL[:,1])
    sofar
end

@everywhere function score_possible_additions(LL, target, end_point)
    -pairwise(Euclidean(), LL, (target-end_point)'');
end   

@everywhere  @inline function fitness(target, end_point)
    #Fitter is larger
    -euclidean(end_point, target)   
end

@everywhere function greedy_addition(LL::Matrix{Float64},
                         target::Vector{Float64},
                         best_word_set::Vector{Int},
                         max_additions = Inf)
    end_point = get_end(LL, best_word_set)
    best_score = fitness(target, end_point)
    
    did_improve = true
    cur_additions = 0
    while(did_improve && cur_additions<max_additions)
        cur_additions+=1
        did_improve=false
        
        addition_scores = score_possible_additions(LL, target, end_point)
        addition_score, addition = findmax(addition_scores)
        if addition_score>best_score+ϵ
            #println("!add: $addition $best_score")
            best_score=addition_score
            best_word_set = [best_word_set...,addition]
            end_point += LL[:,addition]
            did_improve=true
        end
    end
    best_word_set,best_score
end


@everywhere function word_swap_refinement(LL::Matrix{Float64},
                              target::Vector{Float64},
                              best_word_set::Vector{Int})
    
    initial_word_set = copy(best_word_set)
    end_point = get_end(LL, best_word_set)
    best_score = fitness(target, end_point)
    
    function update_best!(word_set,score)
        if score>best_score+ϵ #scores are negative
            best_score=score
            best_word_set = word_set
            #println("*swap, new set: $word_set $score")
        end
    end

    for ii in 1:length(initial_word_set)-1 #Don't need to consider last word added as it was added greedily
        word_set = initial_word_set[[1:ii-1; ii+1:end]]
        sub_endpoint = end_point - LL[:,ii]
        subset_score = fitness(target, sub_endpoint)
        update_best!(word_set, subset_score)

        add_word_set, add_score = greedy_addition(LL, target, word_set, 1) #Try adding just one greedily
        update_best!(add_word_set, add_score)
    end

    best_word_set,best_score
        
end




@everywhere function greedy_search(data::Dict, target::Vector{Float64}; rounds=1000, log=false)
    get_words(word_iis) = [data["indexed_words"][ii] for ii in word_iis]
    
    word_iis = Int[]
    best_score=-Inf
    for round in 1:rounds
        word_iis, add_score = greedy_addition(data["LL"], target, word_iis)
        log && println("POST_ADD_STEP: $add_score $(get_words(word_iis))")
        @assert add_score + ϵ >= best_score
        
        
        if add_score>= 0.0 
            best_score = add_score
            break 
        end        
        
        
        double_word_delete_refinement(data["LL"], target_sowe, best_wordset)
        
        
        word_iis, swap_score = word_swap_refinement(data["LL"], target, word_iis)
        log && println("POST_SWAP_STEP: $swap_score $(get_words(word_iis))")
        @assert swap_score + ϵ >= add_score
        
        
        if best_score - ϵ<swap_score<best_score + ϵ || swap_score>=0.0
            break 
        end       
        
        best_score=swap_score
    end
    get_words(word_iis),best_score,word_iis
end



@everywhere function greedy_search{S<:AbstractString}(data::Dict, target_sent::Vector{S}; kwargs...)
    target::Vector{Float64} = lookup_sowe(data,target_sent)
    greedy_search(data, target; kwargs...)
end

In [86]:
target_sent = test_set[664+6]
target_sowe = lookup_sowe(data,target_sent)
println(target_sent)
println("-------------")
best_words, best_score, best_wordset  = greedy_search(data, target_sent, log=true)


ASCIIString["--","the","president","of","the","kansas","city","local","of","the","international","association","of","fire","fighters","was","severly","injured","today","when","a","bomb","tore","his","car","apart","as","he","left","home","for","work","."]
-------------
POST_ADD_STEP: -11.692032170639072 Any["the","his","people","the","he","the","city","for","people","after","car","of","international","been","fire","his","a","left","the","president","bomb","--","injured","association","as",".","kansas","apart","fighters","tore","works","abundance","appliance","headmaster","gearing","3:30","notably","snuffer","stafford"]
POST_SWAP_STEP: -11.626240963339011 Any["the","people","the","he","the","city","for","people","after","car","of","international","been","fire","his","a","left","the","president","bomb","--","injured","association","as",".","kansas","apart","fighters","tore","works","abundance","appliance","headmaster","gearing","3:30","notably","snuffer","stafford","of"]
POST_ADD_STEP: -1

(Any["the","he","the","city","for","people","after","car","of","international"  …  "notably","of","severly","was","home","proprietor","that","metalsmiths","lastly","of"],-8.889973834977265,[1,17,1,111,9,68,48,558,4,142  …  3540,4,37511,14,159,17509,11,39706,20449,4])

0


LoadError: LoadError: BoundsError: attempt to access 300x40482 Array{Float64,2}:
  0.04656    -0.25539   -0.12559   -0.076947  …   0.42868     0.1722     0.0
  0.21318    -0.25723    0.01363   -0.021211     -0.34601     0.37112    0.0
 -0.0074364   0.13169    0.10306    0.21271       0.05546     0.11523    0.0
 -0.45854    -0.042688  -0.10123   -0.72232      -0.15585     0.043362   0.0
 -0.035639    0.21817    0.098128  -0.13988       0.54724     0.32791    0.0
  0.23643    -0.022702   0.13627   -0.12234   …   0.21315     0.0099291  0.0
 -0.28836    -0.17854   -0.10721   -0.17521       0.3294      0.73261    0.0
  0.21521     0.10756    0.23697    0.12137      -0.16472    -0.62959    0.0
 -0.13486     0.058936   0.3287    -0.070866     -0.043541   -0.27203    0.0
 -1.6413     -1.3854    -1.6785    -1.5721        0.87545     0.41709    0.0
 -0.26091     0.58509    0.22393   -0.22464   …   0.0062048  -0.19369    0.0
  0.032434    0.036501   0.12409    0.04269      -0.020974   -0.57003   

In [369]:
# Shorter Longer Refinement, for replaceing n words with m other words
# Either the words in our current set are too short, so we can limit our search to words that are longer than the sum of our shortest
# Or too long then we do the reverse

#We are removing something, and planning to get closer to the path by adding something
#things longer by adding a finite number more elements
function make_longer_refinement2(LL::Matrix{Float64},
                              target::Vector{Float64},
                              best_word_set::Vector{Int},
                              remove_indexes::Vector{Int},
                              m_add::Int)
    @assert(m_add>=0)
    
    rem_indexes = trues(best_word_set)
    rem_indexes[remove_indexes] = false
    
    cur_word_set = best_word_set[rem_indexes]
    cur_end_point = get_end(LL, cur_word_set)

    cur_lengths = -score_possible_additions(LL[:,best_word_set], target, get_end(LL, Int[])) |> vec

    length_removed = -fitness(target, get_end(LL,best_word_set)) -fitness(target, cur_end_point)
        
    @printval length_removed
    LL_scores = -score_possible_additions(m_add*LL, target, get_end(LL, Int[]))
    
    valid_replacers = find((m_add-1)*maximum(LL_scores) + LL_scores.>=length_removed)   
    valid_replacers
end



make_longer_refinement2 (generic function with 1 method)

In [379]:
make_longer_refinement2(data["LL"], target_sowe, best_wordset, [1:39;], 2)

length_removed = 

40482-element Array{Int64,1}:
     1
     2
     3
     4
     5
     6
     7
     8
     9
    10
    11
    12
    13
     ⋮
 40471
 40472
 40473
 40474
 40475
 40476
 40477
 40478
 40479
 40480
 40481
 40482

121.3273840091464


In [330]:
data["indexed_words"][40482]

"**START1**"

In [338]:
342/3

114.0

In [238]:
LLscores = -score_possible_additions(3*data["LL"], target_sowe, get_end(data["LL"], Int[]))
find(LLscores.<120)

37800-element Array{Int64,1}:
     1
     2
     3
     4
     5
     6
     7
     8
     9
    10
    11
    12
    13
     ⋮
 40426
 40435
 40443
 40449
 40453
 40455
 40464
 40466
 40469
 40476
 40479
 40482

In [210]:
@everywhere function greedy_addition_with_backtrack_prep(LL::Matrix{Float64},
                         target::Vector{Float64},
                         best_word_set::Vector{Int},
                         keep_per_word = 50,
                         max_additions = Inf, 
                         
    )
    
    end_point = get_end(LL, best_word_set)
    best_score = fitness(target, end_point)
    good_ideas = Set{Int}()
    sizehint!(good_ideas, 10*keep_per_word) #Most sentences in most corpora have at least 10 words
    did_improve = true
    cur_additions = 0
    while(did_improve && cur_additions<max_additions)
        
        cur_additions+=1
        did_improve=false
        
        addition_scores = score_possible_additions(LL, target, end_point)
        
        high_scores = select!(collect(enumerate(addition_scores)), 1:keep_per_word, by=ii_score->ii_score[2])
        
        union!(good_ideas, map(ii_score->ii_score[1],high_scores))
        
        addition,addition_score = high_scores[1]
        if addition_score>best_score+ϵ
            #println("!add: $addition $best_score")
            best_score=addition_score
            best_word_set = [best_word_set...,addition]
            end_point += LL[:,addition]
            did_improve=true
        end
    end
    best_word_set,best_score,good_ideas
end

In [225]:
best_word_set,best_score,good_ideas  = greedy_addition_with_backtrack_prep(data["LL"], target_sowe, Int[], )

(Int64[],-112.43741017416913,Set([39722,19698,27248,38963,28730,5870,11237,14446,1374,15603  …  33604,8873,28208,2164,32058,29913,29931,27591,17344,12881]))

In [ ]:
max

In [226]:
#@pipe good_ideas |> map(println,_)
length(good_ideas)

40000

In [142]:
function word_delete_refinement(LL::Matrix{Float64},
                              target::Vector{Float64},
                              best_word_set::Vector{Int})
    
    initial_word_set = pset(best_word_set)
    initial_end_point = get_end(LL, best_word_set)
    best_score = fitness(target, initial_end_point)
    
    function inner(removed, endpoint)
        remaining =  setdiff(initial_word_set, removed)
        if length(remaining)<35
            return
        end
        
        for del_word in remaining
            new_endpoint = endpoint - LL[:,del_word]
            del_score = fitness(target, new_endpoint)
            
            if del_score>best_score + ϵ
                @inbounds best_word_set = remaining
                best_score = del_score
            end
            
            
            inner(removed, endpoint)
        end
        
    end
    
    inner(pset{Int}(),initial_end_point)

    best_word_set,best_score
        
end

word_delete_refinement (generic function with 1 method)

In [143]:
word_delete_refinement(data["LL"], target_sowe, best_wordset)

LoadError: LoadError: InterruptException:
while loading In[143], in expression starting on line 1

In [137]:
length(best_wordset)

39

In [104]:
a

LoadError: LoadError: MethodError: `getindex` has no method matching getindex(::FunctionalCollections.PersistentList{Int64}, ::Int64)
while loading In[104], in expression starting on line 1

In [50]:
function double_word_swap_refinement(LL::Matrix{Float64},
                              LL2::Matrix{Float64}, LL2_index,
                              target::Vector{Float64},
                              best_word_set::Vector{Int})
    
    initial_word_set = copy(best_word_set)
    initial_end_point = get_end(LL, initial_word_set)
    best_score = fitness(LL, target, best_word_set)
    
    for ii in 1:length(initial_word_set)
        for jj in ii+1:length(initial_word_set)
            @inbounds end_point = initial_end_point - LL[:,initial_word_set[ii]] - LL[:,initial_word_set[jj]]
            addition_score, addition_index = findmax(score_possible_additions(LL2, target, end_point))
            if addition_score>best_score + ϵ
                #println("+= $(LL2_index[addition_index,:])")
                @inbounds word_subset = initial_word_set[[1:ii-1; ii+1:jj-1; jj+1:end]]
                @inbounds best_word_set = [word_subset..., LL2_index[addition_index,:]...]
                best_score = addition_score
            end
        end
    end

    best_word_set,best_score
        
end

function get_LL2(LL)
    len_LL2 = (size(LL,2).^2  + size(LL,2))÷ 2
    LL2_index = Matrix{Int}(len_LL2, 2)
    @printval(len_LL2)
    LL2 = Matrix{Float64}(size(LL,1), len_LL2)
    kk=0
    for ii in 1:size(LL,2)
        for jj in 1:ii
            kk+=1
            LL2_index[kk,:] = [ii,jj]
            LL2[:,kk] = LL[:,ii]+LL[:,jj]
        end
    end
    LL2, LL2_index
end


get_LL2 (generic function with 1 method)

In [51]:
LL2=get_LL2(data["LL"])

len_LL2 = 

LoadError: LoadError: OutOfMemoryError()
while loading In[51], in expression starting on line 1

819416403


In [36]:
data["LL"]

300x40482 Array{Float64,2}:
  0.04656    -0.25539   -0.12559   -0.076947  …   0.42868     0.1722     0.0
  0.21318    -0.25723    0.01363   -0.021211     -0.34601     0.37112    0.0
 -0.0074364   0.13169    0.10306    0.21271       0.05546     0.11523    0.0
 -0.45854    -0.042688  -0.10123   -0.72232      -0.15585     0.043362   0.0
 -0.035639    0.21817    0.098128  -0.13988       0.54724     0.32791    0.0
  0.23643    -0.022702   0.13627   -0.12234   …   0.21315     0.0099291  0.0
 -0.28836    -0.17854   -0.10721   -0.17521       0.3294      0.73261    0.0
  0.21521     0.10756    0.23697    0.12137      -0.16472    -0.62959    0.0
 -0.13486     0.058936   0.3287    -0.070866     -0.043541   -0.27203    0.0
 -1.6413     -1.3854    -1.6785    -1.5721        0.87545     0.41709    0.0
 -0.26091     0.58509    0.22393   -0.22464   …   0.0062048  -0.19369    0.0
  0.032434    0.036501   0.12409    0.04269      -0.020974   -0.57003    0.0
  0.056621   -0.19846   -0.086708  -0.4018      

In [55]:
819416403*50*2/ 1000_000_000

81.9416403

In [ ]:
target_sowe = lookup_sowe(data,target_sent)
@time new_best_words, new_best_score =double_word_swap_refinement(LL, LL2, LL2_index, target_sowe, best_wordset)


In [ ]:
target_sent = test_set[12]
println(target_sent)
target_sowe = lookup_sowe(data,target_sent)

In [ ]:
for ii in 1:10
    sample_sowe = (randn!(similar(target_sowe)) + target_sowe)
    print(greedy_search(data, sample_sowe, 1,100, log=false)[1])
    println(",")
end

In [ ]:
res = pmap([1:length(test_set);], test_set) do ii,target_sent
    sol, score = greedy_search(data, target_sent, 1, 100, log=false)
    (sol, score, ii)
end

In [ ]:
open("atis2_res_glove.jsz","w") do fh
    serialize(fh, (Vector{ASCIIString}[sol for (sol, score, ii) in res], test_set)
    )    
end



In [ ]:
se = lookup_sowe(atis_data, ["flights","serve","lunch","which"])
se=se*0.6
greedy_search(atis_data, se, 1, 5, log=false)

In [ ]:
hard_cases = find(x->x[2]<0, res)
res[hard_cases]

In [ ]:
test_set[hard_cases]

In [ ]:
hard_set =  test_set[hard_cases]

hard_res = pmap([1:length(hard_set);], hard_set) do ii,target_sent
    sol, score = greedy_search(atis_data, target_sent, 2,5, log=false)
    (sol, score, ii)
end

In [ ]:
very_hard_cases = find(x->x[2]<0.0, hard_res)

In [ ]:
factorial(14)

In [ ]:
hard_res[very_hard_cases]

In [ ]:


for (ii,target_sent) in enumerate(test_set)
    sol, score = greedy_search(atis_data, target_sent, log=false)
    if score>0
        print("$ii - ")
        println(join(target_sent, " "))
    end
end
    

In [ ]:
enumerate(test_set) |> collect